In [ ]:
import os
import sys
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
import json
import pandas as pd

def twitter_initialize():
    """ Initialize Twitter Authentication & set up API client """
    
    app_key = 'isZI5zPDzVbLdmTrfdQ5lpxSI'
    app_secret = 'Rx19amuj5PBHue77MxBOjxMJYcYMKtRFuTcMZlc8obUaIdoZ0s'
    oauth_token = '2362854974-99uaqgZTY2MoH7t8CvzMIEyScUcxAymEKDqWT42'
#     oauth_token_secret = 'p2L9iLgvyi3Ow4ntDoGwRpeiE8n2II4EMZ7inzHs2b1Mw'
        
    auth = OAuthHandler(app_key, app_secret)
    auth.set_access_token(oauth_token, oauth_token_secret)
    
    ## Setup twitter API Client ##
    client = API(auth, wait_on_rate_limit=True)
    return client


api = twitter_initialize()

query = 'Vacation OR travel OR holiday OR solotrip OR trip'

listtweets=[]

with open("test.json", 'w') as f:
    for tweet in api.search(q="eiffel", geocode="48.858093,2.294694,1mi", lang="en", result_type="mixed", count=100):
        print(tweet.text)
        listtweets.append(tweet.text)
#         json.dump(tweet._json, f)


#Create a data frame (table):

df = pd.read_json('test.json' ,encoding='utf-8', lines= True)
        
# df.head()


In [1]:
from TwitterSearch import *
import time
import datetime
import json
import pandas as pd
from pandas import *
from statistics import mean 

# Ensure that we can see full tweets
pd.set_option('max_colwidth', 350)

In [2]:


def scrapeTweets (jsonFile, ORkeyword_list, AND_word_list = None,geocode = None, startDate = None, endDate = None):
    try:
        
        ts = TwitterSearch(
            consumer_key = 'bQsdYC4VOaQEYzJc0GlS5Kjox',
            consumer_secret = 'sa3S0K38teo6XYZYitM7nW5fcvh8A64O36zYBD4gJ7Woxqr6ku',
            access_token = '1190054573764820993-9BSNn5ix5WkdoSdHitZcdSxCqZJLIi',
            access_token_secret = 'noOLWmmarQTCBcJnt668pNmRhcfJfQmtIRtSj7V2NbzrG'
         )

        
        tso = TwitterSearchOrder()
        tso.set_count(100)
        tso.set_language('en')                               # Set the language 
        tso.set_keywords(ORkeyword_list, or_operator = True)
        
        if(AND_word_list):
            tso.add_keyword(AND_word_list)
        
        if(geocode):
            lat = geocode[0]
            long = geocode[1]
            radius = geocode[2]
            tso.set_geocode(lat, long, radius, imperial_metric=True)
    
        if(startDate):
            tso.set_since(datetime.date(startDate))
        
        if(endDate):
            tso.set_until(datetime.date(endDate))

        
        f = open(jsonFile, 'a') #opening the file 

        for tweet in ts.search_tweets_iterable(tso): #, callback=my_callback_closure):
#             print(tweet)
            json.dump(tweet, f)
            f.write(",")
        

    except TwitterSearchException as e:
        print(e)

                  

In [93]:
def cleaning(text):

    import string
    exclude = set(string.punctuation)

    import re
    # remove new line and digits with regular expression
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\d', '', text)
    # remove patterns matching url format
    url_pattern = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?'
    text = re.sub(url_pattern, ' ', text)
    # standardize white space
    text = re.sub(r'\s+', ' ', text)
    #remove white space
    text = text.strip()

    return text

In [116]:
#Tweet information : Returns a Dataframe 

def parse_tweets(all_tweets):
    #General information regarding each tweet.
    tweetid =  [tweet['id'] for tweet in all_tweets]
    time = [pd.to_datetime(tweet['created_at']) for tweet in all_tweets]
    text = [tweet['text'] if 'text' in tweet else tweet['full_text'] for tweet in all_tweets]
    Hashtags = [tweet['entities']['hashtags'] for tweet in all_tweets]
    source= [tweet['source'] for tweet in all_tweets]

    #User profile information
    User_allinfo = [tweet['user'] for tweet in all_tweets]

    Username = [tweet['user']['name'] for tweet in all_tweets]
    Screen_name = [tweet['user']['screen_name'] for tweet in all_tweets]

    User_location = [tweet['user']['location'] for tweet in all_tweets]
    followers_count = [tweet['user']['followers_count'] for tweet in all_tweets]
    friends_count = [tweet['user']['friends_count'] for tweet in all_tweets]
    Geo_enabled = [tweet['user']['geo_enabled'] for tweet in all_tweets]

    #Tweet location : place 
    Geo = [tweet['geo'] for tweet in all_tweets]
    Coordinates = [tweet['coordinates'] for tweet in all_tweets]
    Place = [tweet['place'] for tweet in all_tweets]

    TravelTweets = pd.DataFrame({'Time':time,'source':source,'Username':Username,'Screen_name':Screen_name,'User_location':User_location,'Text':text,'Hashtags':Hashtags,'Geo_enabled':Geo_enabled,'Geo':Geo,'Coordinates':Coordinates,'Place':Place,'followers_count':followers_count,'friends_count':friends_count,'User_allinfo':User_allinfo}, index=tweetid) #.sort_index(ascending=True)
#     print(TravelTweets)

    cleaner = lambda x: cleaning(x)
    TravelTweets['Text_Clean'] = TravelTweets['Text'].apply(cleaner)
    # Replace and remove empty rows
    TravelTweets['Text_Clean'] = TravelTweets['Text_Clean'].replace('', np.nan)
    TravelTweets = TravelTweets.dropna(how='any')

    return TravelTweets

In [95]:
#Reading all the tweets from the json file.

def extractFromJson(filename) :
    with open(filename,"r") as read_file:
        all_tweets = json.load(read_file)
        return all_tweets

In [96]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentences):
    compound = []
    score =  [analyser.polarity_scores(tweet) for tweet in sentences]
    for sc in score:
        compound.append(sc['compound'])
    
    return compound
#     score_compound = score['compound']
#     print("{:-<300} {}".format(tweet, str(score)))
    


In [6]:
#Getting the final (mean) sentiment score associated with each tweet. 

def Final_sentiment_score(df): #Pass the text column of the dataframe you want to analyze.
    compound_score = sentiment_analyzer_scores(df['Text'])

    df['Compound_score'] = compound_score 

    #Splitting the score into ordinal categories [1,2,3,4,5] : 1,2 <- negetive | 3 <- Neutral | 5 <- Positive
    df['Score'] = pd.cut(df['Compound_score'], bins=5, labels=[1, 2, 3, 4, 5])

    #Converting the tweets sentiment into numerical values. and taking the mean for the complete set.
    df['Score'] = df['Score'].astype('int')

    return df['Score'].mean()


In [97]:

def classification_based_score(df):
    pos_word_score = pd.DataFrame({'pos_score':df.loc[df['Score'] > 3, 'Score']})
    # print(pos_word_score)

    pos_word_score['pos_score'] = pos_word_score['pos_score'].astype('int')

    print("Positive sentiment : ",pos_word_score['pos_score'].mean())
    print("Positive tweets : ",pos_word_score['pos_score'].count())


    neg_word_score = pd.DataFrame({'neg_score':df.loc[df['Score'] < 3, 'Score']})
    # print(pos_word_score)

    neg_word_score['neg_score'] = neg_word_score['neg_score'].astype('int')

    print("Negative sentiment : ",neg_word_score['neg_score'].mean())
    print("Negative tweets : ",neg_word_score['neg_score'].count())


    neu_word_score = pd.DataFrame({'neu_score':df.loc[df['Score'] == 3, 'Score']})
    # print(pos_word_score)

    neu_word_score['neu_score'] = neu_word_score['neu_score'].astype('int')

    print("Neutral sentiment : ",neu_word_score['neu_score'].mean())
    print("Neutral tweets : ",neu_word_score['neu_score'].count())
    
    
# classification_based_score(Barcelona_Tweet)

In [98]:
def classification_based_tweet(df):
    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]


    # Retrevial : show tweets based on +ve or -ve sentiment.

    pos_word_list.append(df.loc[df['Score'] ==5 , 'Text'])
    print("positive" ,pos_word_list[:1])


#     neu_word_list.append(df.loc[df['Score'] == 3, 'Text'])
#     print("Neutral",neu_word_list[:3])

    neg_word_list.append(df.loc[df['Score'] == 1, 'Text'])
    print("Negative",neg_word_list[:1])

In [ ]:
#Destination : 

# scrapeTweets (jsonFile, ORkeyword_list, AND_word_list = None,geocode = None, startDate = None, endDate = None):


In [70]:

for i in range(0,20):
    #Destination: Sagrada Família church
#     scrapeTweets("Barca.json", ['barcelona', 'spain', 'gaudi','Sagrada Família church', 'picasso''espana', 'seafood', 'tapas'],[],[41.40,2.17,4])

    #Destination: 9/11 Memorial
#     scrapeTweets("911_memorial.json", ['9/11', 'Memorial Museum', '9/11 Memorial Museum','grief', 'remembrance','victim','sad','remember','memory'],[],[40.7115,-74.012725,2])

    #Destination: Burj Khalifa 
#     scrapeTweets("Burj_khalifa.json", ['burj khalifa', 'burj', 'khalifa'],[],[25.1972,55.2744,3])

#     #Destination: Auschwitz-Birkenau State Museum, Poland
#     scrapeTweets("Poland.json", ['auschwitz', 'birkenau', 'nazi','concentration', 'camp','auswitz','auswich', 'jewish','Auschwitz-Birkenau State Museum'],[],[50.0348,19.1737,4])

#     #Destination : Sydney Opera House
#     scrapeTweets("Sydney_Opera.json", ['Sydney opera', 'opera', 'mystical'],[],[-33.856159,151.215256,2])

#     #Destination : Taj Mahal
#     scrapeTweets("Taj_mahal.json", ['taj', 'mahal', 'marble','maosoleum', 'jahan','agra', 'emperor'],[],[27.1751,78.0421,4])

#     #Destination : Old Havana
#     scrapeTweets("Havana.json", ['old havana', 'habana', 'vieja''fortress','casa', 'havana'],[],[23.1344,-82.3533,5])

#     #Destination : Golden Gate Bridge
#     scrapeTweets("Golden_Gate.json", ['SF', 'San Fran', 'Golden Gate Bridge','Bridge','bay area', 'san francisco','bay'],[],[37.8199,-122.4783,5])

#     #Destination : Saint Basil's Cathedral, Russia
#     scrapeTweets("Saint_Basil.json", ['moscow', 'st basil\'s', 'saint basil\'s','cathedral', 'red square','church'],[],[55.7525,37.6231,4])

#     #Destination : Yellowstone National Park
#     scrapeTweets("YellowStone.json", ['yellow stone', 'national', 'park','habitat', 'nature','old', 'faithful', 'landscape','wyoming'],[],[44.428,-110.5885,5])


In [29]:
#location and their sentiment dict.

loc_sentiment = {}

In [115]:
Barcelona_Tweet = extractFromJson("Barca.json")

# takes the content and returns a dateframe of relevant columns.
Barcelona_Tweet = parse_tweets(Barcelona_Tweet)


len(Barcelona_Tweet)
# barca_mean = Final_sentiment_score(Barcelona_Tweet)

# print(barca_mean)

# loc_sentiment['Sagrada Família church'] = barca_mean

# classification_based_score(Barcelona_Tweet)

# classification_based_tweet(Barcelona_Tweet)

958

In [100]:
NYC_memorial_tweets = extractFromJson("911_memorial.json")

# takes the content and returns a dateframe of relevant columns.
NYC_memorial_tweets = parse_tweets(NYC_memorial_tweets)

memorial_mean = Final_sentiment_score(NYC_memorial_tweets)

print(memorial_mean)

loc_sentiment['9/11 Memorial'] = barca_mean

classification_based_score(NYC_memorial_tweets)


3.493827160493827
Positive sentiment :  4.466666666666667
Positive tweets :  1500
Negative sentiment :  1.4210526315789473
Negative tweets :  380
Neutral sentiment :  3.0
Neutral tweets :  1360


In [101]:
Burj_khalifa_tweets = extractFromJson("Burj_khalifa.json")

# takes the content and returns a dateframe of relevant columns.
Burj_khalifa_tweets = parse_tweets(Burj_khalifa_tweets)

Burj_mean = Final_sentiment_score(Burj_khalifa_tweets)

print(Burj_mean)

classification_based_score(Burj_khalifa_tweets)

2.7636363636363637
Positive sentiment :  4.4
Positive tweets :  225
Negative sentiment :  1.903225806451613
Negative tweets :  465
Neutral sentiment :  3.0
Neutral tweets :  135


In [102]:
Poland_tweets = extractFromJson("Poland.json")

# takes the content and returns a dateframe of relevant columns.
Poland_tweets = parse_tweets(Poland_tweets)

Poland_mean = Final_sentiment_score(Poland_tweets)

print(Poland_mean)

classification_based_score(Poland_tweets)

2.9941520467836256
Positive sentiment :  5.0
Positive tweets :  85
Negative sentiment :  1.0116279069767442
Negative tweets :  86
Neutral sentiment :  nan
Neutral tweets :  0


In [103]:
Sydney_Opera_tweets = extractFromJson("Sydney_Opera.json")

# takes the content and returns a dateframe of relevant columns.
Sydney_Opera_tweets = parse_tweets(Sydney_Opera_tweets)

Sydney_Opera_mean = Final_sentiment_score(Sydney_Opera_tweets)

print(Sydney_Opera_mean)

classification_based_score(Sydney_Opera_tweets)

3.4048582995951415
Positive sentiment :  4.5
Positive tweets :  320
Negative sentiment :  1.0
Negative tweets :  40
Neutral sentiment :  3.0
Neutral tweets :  628


In [104]:
Taj_mahal_tweets = extractFromJson("Taj_mahal.json")

# takes the content and returns a dateframe of relevant columns.
Taj_mahal_tweets = parse_tweets(Taj_mahal_tweets)

Taj_mahal_mean = Final_sentiment_score(Taj_mahal_tweets)

print(Taj_mahal_mean)

classification_based_score(Taj_mahal_tweets)

2.8095238095238093
Positive sentiment :  4.117647058823529
Positive tweets :  476
Negative sentiment :  1.826086956521739
Negative tweets :  644
Neutral sentiment :  3.0
Neutral tweets :  56


In [105]:
Havana_tweets = extractFromJson("Havana.json")

# takes the content and returns a dateframe of relevant columns.
Havana_tweets = parse_tweets(Havana_tweets)

Havana_mean = Final_sentiment_score(Havana_tweets)

print(Havana_mean)

classification_based_score(Havana_tweets)

3.335458558116724
Positive sentiment :  4.632390745501286
Positive tweets :  778
Negative sentiment :  1.5459057071960298
Negative tweets :  403
Neutral sentiment :  3.0
Neutral tweets :  858


In [106]:
Golden_Gate_tweets = extractFromJson("Golden_Gate.json")

# takes the content and returns a dateframe of relevant columns.
Golden_Gate_tweets = parse_tweets(Golden_Gate_tweets)

Golden_Gate_mean = Final_sentiment_score(Golden_Gate_tweets)

print(Golden_Gate_mean)

classification_based_score(Golden_Gate_tweets)

2.404255319148936
Positive sentiment :  4.411764705882353
Positive tweets :  340
Negative sentiment :  1.9333333333333333
Negative tweets :  1500
Neutral sentiment :  3.0
Neutral tweets :  40


In [107]:
Saint_Basil_tweets = extractFromJson("Saint_Basil.json")

# takes the content and returns a dateframe of relevant columns.
Saint_Basil_tweets = parse_tweets(Saint_Basil_tweets)

Saint_Basil_mean = Final_sentiment_score(Saint_Basil_tweets)

print(Saint_Basil_mean)

classification_based_score(Saint_Basil_tweets)

2.725663716814159
Positive sentiment :  4.444444444444445
Positive tweets :  198
Negative sentiment :  1.775
Negative tweets :  360
Neutral sentiment :  3.0
Neutral tweets :  7


In [108]:
YellowStone_tweets = extractFromJson("YellowStone.json")

# takes the content and returns a dateframe of relevant columns.
YellowStone_tweets = parse_tweets(YellowStone_tweets)

YellowStone_mean = Final_sentiment_score(YellowStone_tweets)

print(YellowStone_mean)

classification_based_score(YellowStone_tweets)

1.875
Positive sentiment :  4.833333333333333
Positive tweets :  120
Negative sentiment :  1.0
Negative tweets :  560
Neutral sentiment :  3.0
Neutral tweets :  120


#### Retrival system : 

In [ ]:
import matplotlib.pyplot as plt 
import IPython.display as anime 
import time

WIDTH = 720 # image (world.png) width
HEIGHT = 360 # image height
DPI = 60 # display image at this dots-per-inch resolution

img = plt.imread("world.jpg")
plt.figure(figsize=(WIDTH/DPI,HEIGHT/DPI))
plt.axis('off')

PopulationListObj = open("cities_population.txt","r")

MyThirtyList = [] #List to store the top 30 cities based on Population.
for i in range(0, 30):
    MyThirtyList.append(PopulationListObj.readline()) #Populating the list reading through the file.

    
#Main loop : To display the image and plot points iterating through the city list.
#Animation of plotting the most populated cities on a map.
for i in range(0, 30):  
    plt.figure(figsize=(WIDTH/DPI,HEIGHT/DPI))
    plt.imshow(img) # Display the image
    plt.axis('off')
    time.sleep(0.5)
    
    #Inner loop : To plot the 'blue' dots that presists previously iterated elements from the city list.  
    for j in range(i):  #Run only till the last 'red' point was plotted.
        BluVar = MyThirtyList[j].strip().split(",")
        BluX,BluY = longlat_to_pixel(float(BluVar[2]), float(BluVar[3])) #Calculating the co-ordinates for a city.
        plt.plot(BluX, BluY, 'bo')
        
    var = MyThirtyList[i].strip().split(",")
    x,y = longlat_to_pixel(float(var[2]), float(var[3]))
    plt.plot(x, y, 'ro') #Plots the 'red' dot
    plt.annotate(var[0], (x+5, y-5)) #Displays the name of the city being plotted on the image.
    plt.show()

    anime.clear_output(wait = True) #Clears the output of the current plot point being displayed.

